# **Esercizi**
## *Esercitazione su Scikit Learn*

In [1]:
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.datasets import load_diabetes, load_iris
from sklearn.feature_selection import (
    SelectKBest,
    mutual_info_regression,
    mutual_info_classif)
from sklearn.linear_model import (
    LinearRegression,
    LogisticRegression)
from sklearn.metrics import (
    adjusted_rand_score,
    mean_absolute_percentage_error,
    precision_score,
    recall_score)
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import (
    DecisionTreeClassifier,
    DecisionTreeRegressor)

## **Es 9.0**

Caricare i dataset Diabetes (a scopo di *regressione*) ed Iris (a scopo di *classificazione*). Visualizzarne rapidamente la struttura.

- Studiare le feature e la distribuzione delle stesse del dataset Diabetes.
- Effettuare un'analisi delle **cross-correlazioni** presenti tra le feature del dataset Diabetes. Utilizzare il $\tau$ di Kendall.
- Isolare le $k$ feature più importanti del dataset Diabetes. Per farlo, utilizzare un oggetto di classe `SelectKBest()` scegliendo la metrica più appropriata tra `mutual_info_regression` e `mutual_info_classif`.
- Comparare i risultati di regressione ottenuti da un regressore lineare e da un albero decisionale in termini di *MAPE*.
- Provare ad eseguire una `GridSearchCV` sull'albero decisionale, e valutare se i risultati in termini di MAPE migliorano.

Ripetere gli stessi step degli esercizi precedenti per il dataset Iris.

- Comparare, per mezzo di due pipeline, i risultati ottenuti effettuando il **clustering** mediante algoritmo **KMeans** sui dati di Iris con e senza *feature selection*.